In [51]:
from selenium import webdriver
from threading import Thread
from tqdm import tqdm
import pandas as pd
import requests
import json
import time

In [52]:
class scrape_ad(Thread):
    def __init__(self, driver, url):
        Thread.__init__(self)
        self.driver = driver
        self.url = url

    def run(self):
        data = None
        try:
            driver.get(url)
            data = pd.read_html(driver.page_source)
        except:
            time.sleep(3)
            try:
                driver.get(url)
                data = pd.read_html(driver.page_source)
            except:
                print(f"Failed to scrape this ad: {url}")
        if data is not None:
            data = pd.concat(data, axis=0).reset_index(drop=True)
            data = data.set_index(0)
            data.columns = [url]
            data = data[~data.index.duplicated(keep='first')]
            data = data.T
            columns = [col for col in data.columns if isinstance(col, str) and 'Insure this property' not in col]
            df = pd.concat([df, data[columns]])

In [ ]:
driver = webdriver.Chrome()
list_urls = set()

# load dataframe from existing csv file or build new one from scratch
try:
    df = pd.read_csv("data/dump_thread.csv").set_index("Unnamed: 0")
except:
    df = pd.DataFrame()

# set page number
n = 1

while True:
    try:
        r = requests.get(f"https://www.immoweb.be/en/search-results/house-and-apartment/for-rent?countries=BE&page={n}&orderBy=newest")
        list_ads = r.json()["results"]
        for ad in list_ads:
            property_type = ad["property"]["type"]
            locality = ad["property"]["location"]["locality"]
            postalcode = ad["property"]["location"]["postalCode"]
            id = ad["id"]
            url = f"https://www.immoweb.be/en/classified/{property_type}/for-rent/{locality}/{postalcode}/{id}"
            list_urls.add(url)
        
        for url in list_urls:
            thread = scrape_ad(driver, url)
            thread.start()
    
        # store dataframe in csv file
        df.to_csv("data/dump_thread.csv")

        # go to next page
        n += 1
        
    except:
        break

    # test on a limited number of pages
    if n == 2:
        break

driver.close()
pd.read_csv("data/dump_thread.csv").shape

In [56]:
list_urls = [
    "https://www.immoweb.be/en/classified/APARTMENT/for-rent/Woluwe-Saint-Lambert/1200/10006280",
    "https://www.immoweb.be/en/classified/HOUSE/for-rent/Braine-l'Alleud/1420/9871444",
    "https://www.immoweb.be/en/classified/APARTMENT/for-rent/LONZEE/5030/10006290"
]

driver = webdriver.Chrome()
df = pd.DataFrame()

for url in list_urls:


    thread = scrape_ad(driver, url)
    thread.start()


    # data = None
    # try:
    #     driver.get(url)
    #     data = pd.read_html(driver.page_source)
    # except:
    #     time.sleep(3)
    #     try:
    #         driver.get(url)
    #         data = pd.read_html(driver.page_source)
    #     except:
    #         print(f"Failed to scrape this ad: {url}")
    # if data is not None:
    #     data = pd.concat(data, axis=0).reset_index(drop=True)
    #     data = data.set_index(0)
    #     data.columns = [url]
    #     data = data[~data.index.duplicated(keep='first')]
    #     data = data.T
    #     columns = [col for col in data.columns if isinstance(col, str) and 'Insure this property' not in col]
    #     df = pd.concat([df, data[columns]])

driver.close()
df.shape
df

""


Failed to scrape this ad: https://www.immoweb.be/en/classified/APARTMENT/for-rent/LONZEE/5030/10006290
Failed to scrape this ad: https://www.immoweb.be/en/classified/APARTMENT/for-rent/LONZEE/5030/10006290
Failed to scrape this ad: https://www.immoweb.be/en/classified/APARTMENT/for-rent/LONZEE/5030/10006290
